## A guide on how to use Orientdb!
This notebook demonstrate on how to use pyorient- a python driver for orientdb to communicate with the orientdb.


## Some points to be noted!
* Before setting up this notebook, set up orientdb on kubernetes.

* To set up the orientdb on kubernetes, follow this developer journey : https://github.com/IBM/deploy-graph-db-container.

* The notebook makes use of a config.json file which include all the details about the database you want to create on orientdb like username, password of the orientdb, and the schema for your database.

* Its recommended to set the same username and password for databases that you have set for the orientdb for the ease.In the yaml file for creating the orientdb service on kubernetes, the username and the password used is root and  rootpwd. However, user can change it as per their choice in yaml file and the config file.

* The dataset consumed by this notebook is kaggle imdb movie dataset, you can download it from https://www.kaggle.com/deepmatrix/imdb-5000-movie-dataset.

* For the purpose of this tutorial, data has been trimmed to 600 rows and named it as `graphdb-insights.csv`.

* This notebook gives hands on how to perform various operations on orientdb through python.

* Orientdb also provides an interactive orientdb studio to create new graph database, create schema, add, delete, and update records.You can watch this video tutorial https://www.youtube.com/watch?v=l-OVSjf-vk0&feature=youtu.be to know the working of orientdb.You can follow the documentation(readme file) of this project which give a comprehensive tutorial on how to use orientdb console. 



### Install the pyorient package!

In [22]:
######## download pyorientdb 
! pip install pyorient --user 

## Import packages and libraries.

In [23]:
import pyorient, json, pandas as pd
import sys

### Add your service credentials for Object Storage
* You must create Object Storage service on Bluemix. To access data in a file in Object Storage, you need the Object Storage authentication credentials. Click on insert to code for the file you want to use in the code.
* For config.json file, click on insert to code dropdown and choose `insert credentials`.
* For movie dataset ( graphdb-insights.csv ), click on insert to code dropdown and choose `Insert Pandas Dataframe`

In [24]:
# # @hidden_cell
credentials_1 = {
  'auth_url':'https://identity.open.softlayer.com',
  'project':'object_storage_3fd48083_3bbd_4355_a296_f2558dc1b897',
  'project_id':'07704599ed444df0927832e455d9ec63',
  'region':'dallas',
  'user_id':'fb16c3d0955c42538faa723269123f5d',
  'domain_id':'1bfd0aa070764f39845124cf77d75a19',
  'domain_name':'1104675',
  'username':'member_b6c199f708035093454e23f2785a418ef096423b',
  'password':"""VO~S.?)5E6g2!!Dh""",
  'container':'graphdb',
  'tenantId':'undefined',
  'filename':'config.json'
}

from io import BytesIO  
import requests  
import json  
import pandas as pd

# loading the config.json

def get_data(credentials):  
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage V3."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': credentials['username'],'domain': {'id': credentials['domain_id']},
            'password': credentials['password']}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', credentials['container'], '/', credentials['filename']])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return json.loads(resp2.content)


# loading the imdb movie data

from io import StringIO
import requests
import json
import pandas as pd

# @hidden_cell
# This function accesses a file in your Object Storage. The definition contains your credentials.
# You might want to remove those credentials before you share your notebook.
def get_object_storage_file_with_credentials_3fd480833bbd4355a296f2558dc1b897(container, filename):
    """This functions returns a StringIO object containing
    the file content from Bluemix Object Storage."""

    url1 = ''.join(['https://identity.open.softlayer.com', '/v3/auth/tokens'])
    data = {'auth': {'identity': {'methods': ['password'],
            'password': {'user': {'name': 'member_b6c199f708035093454e23f2785a418ef096423b','domain': {'id': '1bfd0aa070764f39845124cf77d75a19'},
            'password': 'VO~S.?)5E6g2!!Dh'}}}}}
    headers1 = {'Content-Type': 'application/json'}
    resp1 = requests.post(url=url1, data=json.dumps(data), headers=headers1)
    resp1_body = resp1.json()
    for e1 in resp1_body['token']['catalog']:
        if(e1['type']=='object-store'):
            for e2 in e1['endpoints']:
                        if(e2['interface']=='public'and e2['region']=='dallas'):
                            url2 = ''.join([e2['url'],'/', container, '/', filename])
    s_subject_token = resp1.headers['x-subject-token']
    headers2 = {'X-Auth-Token': s_subject_token, 'accept': 'application/json'}
    resp2 = requests.get(url=url2, headers=headers2)
    return StringIO(resp2.text)




## Load the config file.
 

In [25]:
node_data = get_data(credentials_1)
print node_data

{u'username': u'root', u'edge_class': {u'acted_in': {u'out': {u'Linked_Class': u'person', u'Type': u'Link'}, u'in': {u'Linked_Class': u'movie', u'Type': u'Link'}}, u'worked_with': {u'out': {u'Linked_Class': u'person', u'Type': u'Link'}, u'in': {u'Linked_Class': u'person', u'Type': u'Link'}}}, u'Database_name': u'testdb', u'vertex_class': {u'person': {u'role': u'String', u'name': u'String', u'fblikes': u'Float'}, u'movie': {u'title': u'String', u'numCriticForReviews': u'Integer', u'imdbRating': u'Float', u'durationInMins': u'Integer', u'year': u'Integer', u'genre': u'String', u'movieFacebookLikes': u'Float', u'plotKeywords': u'String'}}, u'host': u'localhost', u'password': u'root', u'port': 2424}


### Load the movie data.
* There are many rows and columns in the data that are empty. Hence, It is important to clean the data. 
* All the empty rows and columns are dropped from the dataframe using `dropna()` function of pandas.

In [ ]:
imdb_df= pd.read_csv(get_object_storage_file_with_credentials_3fd480833bbd4355a296f2558dc1b897('graphdb', 'INSIGHTS-GRAPHDB.csv'))
imdb_df = imdb_df.dropna()
imdb_df.head(10)

##  Connecting to Orientdb
* Please use the 2424/node port for 2424 to access orientdb. The other port 2480/nodeport for 2480 is for orientdb studio.
* You can refer to the readme of https://github.com/IBM/deploy-graph-db-container to find the ip-address of the kubernetes cluster and node port to which 2480 has been mapped.
* Uncomment the first line of the next cell and Replace the content in the angular brackets and run this to connect to the orientdb that you have set up on kubernetes.
`client = pyorient.OrientDB(<ip-address-of-the-kubernetes-cluster>,<node-port mapped to port 2424 of orientdb>)`

In [28]:
#client = pyorient.OrientDB(<ip-address-of-the-kubernetes-cluster>,<node-port mapped to port 2424 of orientdb>)

print(client)

### session-id username and password is global paswword, you have set for orientdb
session_id = client.connect(node_data['username'], node_data['password'])





## Some handy functions used in the notebook.
#### verifying functions to avoid duplicacy errors!
*  method to check if class is present or not 
*  method to check if a person is present or not 
*  method to check if a movie is present or not 
#### To get information of a class
*  method to find the cluster of a class


In [29]:
# method to check if class is present or not 
def check_if_class_present_or_not(classname):
    name="will be replaced by query result name"
    query = "SELECT FROM ( SELECT expand( classes ) FROM metadata:schema ) WHERE name = " +'"'+ classname + '"'
    print(query)
    a = client.command(query)
    
    for k in a:
        name = k.name
    
    if(name == classname):
        return True
    else:
        return False
    
# method to check if a person is present or not     
def check_if_already_present():
    check_if_already_present = "select * from person"
    c = client.command(check_if_already_present)
    d=[]

    for name in c:
        d.append([name.name, name.role, name.fblikes])


    check_df = pd.DataFrame(list(d), columns=['name','role','fblikes'])
    return check_df


# method to check if a movie is present or not 
def check_if_already_present_movie():
    check_if_already_present = "select title , year from movie"
    c = client.command(check_if_already_present)
  
    d=[]

    for name in c:
        d.append([name.title, name.year])


    check_df_movie = pd.DataFrame(list(d), columns=['title','year'])
    return check_df_movie

## method to find the cluster of a class 
def find_the_Cluster_id_of_a_class (classname):
    find_the_Cluster_id_of_a_class = "SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = '" + classname + "'"
    print(find_the_Cluster_id_of_a_class)
    c = client.command(find_the_Cluster_id_of_a_class)
    for ids in c:
        return ids.defaultClusterId




#### Creating a new database!
* Method for creating Database : createDatabase() : takes node_data as argument. This method checks if database you want to create is already present  

#### Creating Node(vertex) class
* There will be two vertex classes namely - Person and Movie.With Person’s attribute as Role: Director/Actor, Name, Fb-likes; Movie’s Attributes as Title, Year, IMDB, rating, Duration, Language, Genre, Plot, keywords, Num_critic_for_reviews, movie_facebook_likes.command to create the edge class: `create class <classname> extends V`.
* There are two methods : 
    * createNodeClass_withSchema() : use this method when the user knows the properties of a node class. 
    * createNodeClass_NoSchema() :   use this method when the user doesn't knows the properties of a node class.
* As in the config file, The schema has been defined for the person class and edge class.Function createNodeClass_withSchema() has been used.However, the user can use the other function when they don't have defined schema.

#### Creating Relations(Edge) class
* In Graph databases, a relation is the connection between two nodes, which in OrientDB is called an edge. Edges are bidirectional and can only connect two vertices. With the Graph API, Edges connect only two vertices.This means that 1:n relationships are not allowed. To specify a 1:n relationship with graphs, create multiple edges.command to create the edge class: `create class <classname> extends E`.

* The method used is :
    * createEdgeClass() : Edges have two ends.It always start from one vertex class and ends on another vertex.Its acts as a bridge between two vertices.Hence, The edge will always have in and out property. To create these properties of the Edge class, the commands are 
    `Create property <class-name>.in IF NOT EXISTS Link <linked_vertex_class>`
    `Create property <class-name>.out IF NOT EXISTS Link <linked_vertex_class>`
    


In [30]:
#  Method for creating Database
def createDatabase(node_data):
    # check if it exists or not
    if client.db_exists( node_data['Database_name'], pyorient.STORAGE_TYPE_MEMORY ):
        client.db_open(node_data['Database_name'],node_data['username'], node_data['password'])
        print node_data['Database_name'] + " "+ "has already been created and opened for use"
    else:
    # create the database
        client.db_create( node_data['Database_name'], pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY )
        print node_data['Database_name'] + " created and opened successfully"
        

# Method for createNodeClass_withSchema
def createNodeClass_withSchema(node_data):
    for class_name,value in node_data['vertex_class'].items():
        bool_result = check_if_class_present_or_not(class_name)
        print bool_result
        if(not bool_result):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            print(command_to_create_node)
            cluster_id = client.command(command_to_create_node) 
            print class_name,cluster_id
            for property_name,value in node_data['vertex_class'][class_name].items():
                ### create properties 
                command_to_create_property= "create property"+ " "+ class_name +"." + property_name +" " +"IF NOT EXISTS " + value
                print(command_to_create_property)
                client.command(command_to_create_property) 
            print "class " + class_name + " and its properties successfully created"
        else:
            print "this class is already present !!"

# method for creating createNodeClass_NoSchema.
def createNodeClass_NoSchema(node_data):
    for class_name,value in node_data['vertex_class'].items():
        if(check_if_class_present_or_not(class_name)):
            command_to_create_node = "create class"+" "+ class_name +" "+ "extends V"
            print(command_to_create_node)
            cluster_id = client.command(command_to_create_node) 
            print class_name + " has been created with cluster id " +cluster_id
        else:
            print "this class is already present !!"
            
# method for creating edge class.
def createEdgeClass(node_data):
    for class_name,v in node_data['edge_class'].items():
        if(not check_if_class_present_or_not(class_name)):
            command_to_create_edge_class = "create class"+" "+ class_name +" "+ "extends E"
            print(command_to_create_edge_class)
            cluster_id = client.command(command_to_create_edge_class)
            print("Edge class" +" " + class_name + "has been created successfully!")

            for key,val in node_data['edge_class'][class_name].items():
                print key,val
                command_to_create_property= "create property"+ " "+ class_name +"." + key +" " +"IF NOT EXISTS " + node_data['edge_class'][class_name][key]['Type']+" " + node_data['edge_class'][class_name][key]['Linked_Class']
                print(command_to_create_property)
                client.command(command_to_create_property) 
                print node_data['edge_class'][class_name][key]['Type'], node_data['edge_class'][class_name][key]['Linked_Class']
        else:
             print "this class is already present !!"
            


     

In [35]:
createDatabase(node_data)
createNodeClass_withSchema(node_data)
#createNodeClass_NoSchema(node_data)
createEdgeClass(node_data)

### Creating nodes and edges between them.

#### creating nodes
* Before inserting any record for person or movie, it is checked that if its already present or not using     check_if_already_present().
* check_if_already_present() -- function returns the dataframe with the records of person class already present in the database.
* check_if_already_present_movie() -- function returns the dataframe with the records of a movie class already present in the database.

#### creating relations
* check if both the nodes are present in the records before creating an edge between them.
* In this person to person relation has been created with `worked_with` class and person to movie has been created with `acted_in`


In [32]:
# method to create the edge and the link when you have designed schema for your graphdb
def creating_records(imdb_df):
    for index, row in imdb_df.iterrows():
        ''' creating records -- all three actors and director 
        '''
        check_df = check_if_already_present()
        if(any(check_df.name == row["actor_1_name"])):
            print "node "+row["actor_1_name"] +" is already present!!"
        else:
            command_to_create_actor_1_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_1_name"]+'"' + ','+ str(row["actor_1_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_1_node_class_person)
            print "node "+row["actor_1_name"] +" has been created!!"
            
        if(any(check_df.name == row["actor_2_name"])):
            print "node "+row["actor_2_name"] +" is already present!!"
        else:
            command_to_create_actor_2_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_2_name"]+'"' + ','+ str(row["actor_2_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_2_node_class_person)
            print "node "+row["actor_2_name"] +" has been created!!"
            
        if(any(check_df.name == row["actor_3_name"])):
            print "node "+row["actor_3_name"] +" is already present!!"
        else:
            command_to_create_actor_3_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["actor_3_name"]+'"' + ','+ str(row["actor_3_facebook_likes"])+',' +'"' +'actor'+'"' + ")"
            client.command(command_to_create_actor_3_node_class_person)
            print "node "+row["actor_3_name"] +" has been created!!"
            
        if(any(check_df.name == row["director_name"])):
            print "node "+row["director_name"] +" is already present!!"
        else:
            command_to_create_director_node_class_person  = "INSERT INTO person (name, fblikes, role) VALUES (" +'"' +row["director_name"]+'"' + ','+ str(row["director_facebook_likes"])+',' +'"' +'director'+'"' + ")"
            client.command(command_to_create_director_node_class_person)
            print "node "+row["director_name"] +" has been created!!"
        
        command_to_create_movie  = "INSERT INTO movie (title, year, durationInMins, imdbRating, genre, plotKeywords, numCriticForReviews, movieFacebookLikes) VALUES (" +'"' +row["movie_title"]+'"' + ','+ str(row["title_year"])+','+str(row["duration"])+','+str(row["imdb_score"])+',' +'"'+row["genres"]+'"' +','+'"' +row["plot_keywords"]+'"'+','+str(row["num_critic_for_reviews"])+','+str(row["movie_facebook_likes"])+")"
        print command_to_create_movie
        client.command(command_to_create_movie)
        print "node "+row["movie_title"] +" has been created!!"
        
        

        
        
def createRelationships():
      for index, row in imdb_df.iterrows():
            '''create_edge_between_two_actors_1 and actor_2 
            '''
            if((row["actor_1_name"]  in check_df.name.values) and (row["actor_2_name"]  in check_df.name.values)):
                command_to_create_edge_between_two_actors_1_and_2 = "create edge worked_with from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_1_and_2)
                client.command(command_to_create_edge_between_two_actors_1_and_2)
                print "edge has been created successfully!"
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            
            '''create_edge_between_two_actors_1 and actor_2 
            '''
            if((row["actor_2_name"]  in check_df.name.values) and  (row["actor_3_name"]  in check_df.name.values) ):
                command_to_create_edge_between_two_actors_2_and_3 = "create edge worked_with from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_2_and_3)
                client.command(command_to_create_edge_between_two_actors_2_and_3)
                print "edge has been created successfully!"
            else:
                   print "edge cant be created because vertext is not present !!!!!"
               

            '''create_edge_between_two_actors_1 and actor_2
            '''
            if((row["actor_2_name"]  in check_df.name.values) and ( row["actor_3_name"]  in check_df.name.values )):
                command_to_create_edge_between_two_actors_3_and_1 = "create edge worked_with from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
                print(command_to_create_edge_between_two_actors_3_and_1)
                client.command(command_to_create_edge_between_two_actors_3_and_1)
                print "edge has been created successfully!"
            else:
                   print "edge cant be created because vertext is not present !!!!!"
                
            '''create_edge_between actor_1 and director 
            '''
            if((row["actor_1_name"]  in check_df.name.values) and  (row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_1_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_1_name"]+'"'+")"
                print(command_to_create_edge_between_actor_1_and_director)
                client.command(command_to_create_edge_between_actor_1_and_director)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

            '''create_edge_between actor_2 and director
            '''
            if((row["actor_2_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_2_and_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_2_name"]+'"'+")"
                print(command_to_create_edge_between_actor_2_and_director)
                client.command(command_to_create_edge_between_actor_2_and_director)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            '''create_edge_between actor_3 and director 
            '''
            if((row["actor_3_name"]  in check_df.name.values) and ( row["director_name"] in check_df.name.values) ):
                command_to_create_edge_between_actor_3_director = "create edge worked_with from (select from person where name = "+'"'+row["director_name"]+'"'+ ")"+" " +"to (select from person where name = "+'"'+row["actor_3_name"]+'"'+")"
                print(command_to_create_edge_between_actor_3_director)
                client.command(command_to_create_edge_between_actor_3_director)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

            '''movies and actors 
            '''
            '''create_edge_between_actors_1_and_movie
            '''
            if((row["actor_1_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
                command_to_create_edge_between_actors_1_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_1_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
                print(command_to_create_edge_between_actors_1_and_movie)
                client.command(command_to_create_edge_between_actors_1_and_movie)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

            '''create_edge_between_actors_2_and_movie
            '''
            if((row["actor_2_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values) ):
                command_to_create_edge_between_actors_2_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_2_name"]+'"'+ ")"+" " +"to (select from movie where title = "+'"'+row["movie_title"]+'"'+")"
                print( command_to_create_edge_between_actors_2_and_movie )
                client.command(command_to_create_edge_between_actors_2_and_movie)
            else:
                   print "edge cant be created because vertext is not present !!!!!"
            
            '''create_edge_between_actors_3_and_movie 
            '''
            if((row["actor_3_name"]  in check_df.name.values) and  (row["movie_title"] in check_df_movie.title.values)):
                command_to_create_edge_between_actors_3_and_movie = "create edge acted_in from (select from person where name = "+'"'+row["actor_3_name"]+'"'+ ")"+" " +"to (select from movie where title= "+'"'+row["movie_title"]+'"'+")"
                print(command_to_create_edge_between_actors_3_and_movie)
                client.command(command_to_create_edge_between_actors_3_and_movie)
            else:
                   print "edge cant be created because vertext is not present !!!!!"

        





In [34]:
creating_records(imdb_df)
check_df = check_if_already_present()
check_df_movie = check_if_already_present_movie()
createRelationships()


### The orientdb provides flexible schema as it is a nosql and sql database.
* To create a record using json.
    * use this function of pyorient : 
        ` client.record_create(<cluster_id>, <data>)`
        `Data should be in json format. `


`Note that a  new property  "born_in" which is not defined in schema, has been added in the json.You can add as many number of attributes in the json for a particular record. The record with stored as json.`

In [ ]:
data = {
        'person': {
                "name": "John", 
                "role": "director",
                "fblikes": 400000.0,
                "born_in" : 1980
                },
        }

for key, value in data.iteritems():
        cluster_id = find_the_Cluster_id_of_a_class(key)
        print cluster_id

def creating_records_noschema(data):
        id = client.record_create(cluster_id, data)
        print "record succesfully created with " + str(id)
        
creating_records_noschema(data)


SELECT defaultClusterId from (SELECT expand( classes ) FROM metadata:schema) where name = 'person'


### Most Mentioned
Movie with maximum no of facebook likes.


In [173]:
#most mentioned 
def most_mentioned_movie():
    a = client.command(('select max(movieFacebookLikes) from movie '))

    for max_num in a :
        max_num = max_num.max
    print max_num
    most_mentioned_movie_object = client.command('select title from movie where movieFacebookLikes = ' + str(max_num))
    
    for titles in most_mentioned_movie_object :
        print titles.title

In [174]:
most_mentioned_movie()

349000
InterstellarÂ


### Clustering 

Cluster movies with rating above 7.



In [168]:
def movie_with_imdb_rating_above_7():
    title = []
    a = client.command(('select title from movie where imdbRating > 7 '))
    for titles in a :
        title.append(titles.title)
    title_df = pd.DataFrame(list(title), columns=['title'])    
    return title_df


In [170]:
movie_with_imdb_rating_above_7()